# Create VM in Azure
Follow this notebook to easily create virtual machines in Azure.

### Choose a name for your VM
Windows computer name cannot be more than 15 characters long, be entirely numeric, or contain the following characters: ` ~ ! @ # $ % ^ & * ( ) = + _ [ ] { } \ | ; : . ' " , < > / ?.

In [ ]:
$vmName = "Win11EntN21H2"

### Choose the VM Image
Folllow the instructions on [get-vm-images-from-marketplace.ipynb]("./get-vm-images-from-marketplace.ipynb") to choose an OS Image.

In [ ]:
$imageName = "MicrosoftWindowsDesktop:Windows-11:win11-21h2-ent:22000.978.220910"

### Choose the region where to deploy the VM

In [ ]:
$location = "westus2"

### Choose a size
The size means how many CPUs, how much RAM, the hard drive, etc.

> As of 15th Feb 2022, the links here are kinda useless, just use `Standard_D4s_v4`.

Check out the following link to understand the available VMs and their pricing before deciding for one:

[Windows Virtual Machines Pricing](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/windows/)

In addition, if you need Virtualization (WSL, Docker, etc), you need to check which ones have CPUs compatible with nested virtualization here [Azure compute unit (ACU)](https://docs.microsoft.com/en-us/azure/virtual-machines/acu). Look for trhe ones that have `***`.

Yoy can also search for VM Sizes with the following code:

In [ ]:
Get-AzVmSize -Location $location | Where-Object { $_.Name  -like "*D3*"}

In [ ]:
$vmSize = "Standard_D4s_v4"

### Choose a Resource Group
You will need a Resource group. You can either use an existing Resource Group or create a new one.

 Check the current resource groups:

In [ ]:
Get-AzResourceGroup

If you found a resource group, store it in the following variable.

If you want to create a new resource group, write the name in the following variable and execute the next command.

In [ ]:
$resourceGroupName = "RG-ForVMs"

In [ ]:
New-AzResourceGroup -Name $resourceGroupName -Location $location

### Choose the credentials for the admin user

Password: More 8-123 chars, 1 capital, 1 number, 1 special

In [ ]:
$credentials = Get-Credential

### Configure the parameters for the VM:

In [ ]:
$vmParams = @{
    ResourceGroupName = $resourceGroupName
    Name = $vmName
    Location = $location
    ImageName = $imageName
    PublicIpAddressName = $vmName + "IP"
    Credential = $credentials
    OpenPorts = 3389
    Size = $vmSize
}

$vmParams

### Create the VM:

In [ ]:
$createdVMInfo = New-AzVM @vmParams
$createdVMInfo

To further explore the values returned, follow the instructions here:

[Microsoft Docs | Get VM information with queries](https://docs.microsoft.com/en-us/powershell/azure/azureps-vm-tutorial?view=azps-7.2.0&tutorial-step=5)

## Connect to the VM

In [ ]:
Get-AzRemoteDesktopFile -ResourceGroupName $resourceGroupName -Name $vmName -Launch

### Save the RDP file

In [ ]:
$localPath = "C:\Work\" + $vmName + ".rdp" 
Get-AzRemoteDesktopFile -ResourceGroupName $resourceGroupName -Name $vmName -LocalPath $localPath
Invoke-Item $localPath

In [ ]:
$imageVersions = Get-AzResource -ResourceType Microsoft.Compute/galleries/images/versions
$imageVersions